In [1]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [1]:
!pip install git+https://github.com/afnan47/cuda.git


  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-zns2v0fh
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-zns2v0fh
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=1f04312f70c21259f7aef0d939cc814512a1b13701124cda7f03090615965011
  Stored in directory: /tmp/pip-ephem-wheel-cache-2t0twv9b/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [3]:
%%cu
#include<iostream>
using namespace std;

__global__ void add(int * a, int* b , int *c, int size)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if(i < size){
        c[i] = b[i]+a[i];
    }
}

__managed__ int vector_a[256], vector_b[256], vector_c[256];


int main()
{
    for(int i=0;i<256;i++)
    {
        vector_a[i] = i;
        vector_b[i] = 256-i;

    }
    add<<<1,256>>>(vector_a,vector_b,vector_c,256);
    cudaDeviceSynchronize();

    for(int i=0;i<256;i++)
    {
        cout<<vector_c[i]<<" ";
    }
 return 0;
}

256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 256 

In [5]:
%%cu
#include<iostream>
using namespace std;

__global__ void multiply(int* A, int* B, int* C, int size) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < size && col < size) {
        int sum = 0;
        for (int i = 0; i < size; i++) {
            sum += A[row * size + i] * B[i * size + col];
        }
        C[row * size + col] = sum;
    }
}

__managed__ int matA[9], matB[9], matC[9];  // Corrected to hold 3x3 matrices

void print(int* matrix, int size) {
    for (int row = 0; row < size; row++) {
        for (int col = 0; col < size; col++) {
            cout << matrix[row * size + col] << " ";
        }
        cout << '\n';
    }
    cout << '\n';
}

int main()
{
    int N = 3;  // Size of the matrix (3x3)
    for(int i = 0; i < N*N; i++) {
        matA[i] = rand() % 10 + 1;  // Random values between 1 and 10
        matB[i] = rand() % 10 + 1;
    }

    cout << "Matrix A:" << endl;
    print(matA, N);
    cout << "Matrix B:" << endl;
    print(matB, N);

    // Define the number of threads per block and the number of blocks per grid dimension
    dim3 threads(N, N);  // One thread for each element in the matrix
    dim3 blocks(1, 1);   // Since N is manageable within a single block, we use one block

    multiply<<<blocks, threads>>>(matA, matB, matC, N);
    cudaDeviceSynchronize();  // Wait for the GPU to finish before accessing on host

    cout << "Resulting Matrix C:" << endl;
    print(matC, N);
    return 0;
}


Matrix A:
4 8 4 
7 10 3 
1 4 1 

Matrix B:
7 6 6 
3 2 8 
10 7 7 

Resulting Matrix C:
92 68 116 
109 83 143 
29 21 45 


